In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba

In [4]:
#@numba.jit
def randomWalk1D(numberOfCycles, jumpsPerCycle, p=0.5):
    distribution = np.zeros(2 * jumpsPerCycle + 1)
    positions = np.zeros(jumpsPerCycle)
    msd = np.zeros(jumpsPerCycle)
    
    for cycle in range(numberOfCycles):
        currentPosition = 0
        
        # Make displacements and save to positions array
        for k in range(jumpsPerCycle):
            currentPosition += (1 if np.random.rand() < p else -1)
            positions[k] = currentPosition
        
        # Update MSD and position position distribution
        msd += sampleMSD(positions, jumpsPerCycle)
        distribution[int(currentPosition + jumpsPerCycle)] += 1
        
    # normalize
    distribution /= (numberOfCycles * jumpsPerCycle)
    msd /= numberOfCycles
    return distribution, msd

#@numba.jit
def sampleMSD(positions, jumpsPerCycle):
    # This implementation is post-hoc, which is fine if you can pass an array of positions, 
    # but is intractable for simulations, as we then have to save (T, N, 3) sized position arrays.
    msd = np.zeros(jumpsPerCycle)
    for dt in range(1, jumpsPerCycle):
        msd[dt] = np.mean((positions[dt:] - positions[:-dt])**2)
    return msd
    

In [5]:
numberOfCycles = 100000
jumpsPerCycle = 100
p = 0.5

walk, msd = randomWalk1D(numberOfCycles, jumpsPerCycle, p)

In [ ]:
fig, ax = plt.subplots(2)
ax[0].plot(np.arange(-jumpsPerCycle, jumpsPerCycle + 1), walk)
ax[0].set_xlim(-100, 100)
ax[1].plot(msd)